## Task

In this compulsory task you will clean the country column and parse the date column in the **store_income_data_task.csv** file.

In [ ]:
# Load up store_income_data.csv

In [3]:
import numpy as np
import pandas as pd

# Libraries
import fuzzywuzzy
from fuzzywuzzy import process
import chardet

# Set seed for reproducibility
np.random.seed(0)

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [68]:
income_df = pd.read_csv("store_income_data_task.csv")
income_df.head()

,id,store_name,store_email,department,income,date_measured,country
0,1,"Cullen/Frost Bankers, Inc.",NaN,Clothing,$54438554.24,4-2-2006,United States/
1,2,Nordson Corporation,NaN,Tools,$41744177.01,4-1-2006,Britain
2,3,"Stag Industrial, Inc.",NaN,Beauty,$36152340.34,12-9-2003,United States
3,4,FIRST REPUBLIC BANK,ecanadine3@fc2.com,Automotive,$8928350.04,8-5-2006,Britain/
4,5,Mercantile Bank Corporation,NaN,Baby,$33552742.32,21-1-1973,United Kingdom


1. Take a look at all the unique values in the "country" column. Then, convert the column to lowercase and remove any trailing white spaces.

In [69]:
countries = income_df['country'].unique()
print(f"There are {len(countries)} unique countries")
countries

There are 77 unique countries


array(['United States/', 'Britain', ' United States', 'Britain/',
       ' United Kingdom', 'U.K.', 'SA ', 'U.K/', 'America',
       'United Kingdom', nan, 'united states', ' S.A.', 'England ', 'UK',
       'S.A./', 'ENGLAND', 'BRITAIN', 'U.K', 'U.K ', 'America/', 'SA.',
       'S.A. ', 'u.k', 'uk', ' ', 'UK.', 'England/', 'england',
       ' Britain', 'united states of america', 'UK/', 'SA/', 'SA',
       'England.', 'UNITED KINGDOM', 'America.', 'S.A..', 's.a.', ' U.K',
       ' United States of America', 'Britain ', 'England', ' SA',
       'United States of America.', 'United States of America/',
       'United States.', 's. africasouth africa', ' England',
       'United Kingdom ', 'United States of America ', ' UK',
       'united kingdom', 'AMERICA', 'America ',
       'UNITED STATES OF AMERICA', ' S. AfricaSouth Africa', 'america',
       'S. AFRICASOUTH AFRICA', 'Britain.', '/', 'United Kingdom.',
       'United States', ' America', 'UNITED STATES', 'sa',
       'United States

In [70]:
# Convert to lower case
income_df['country'] = income_df['country'].str.lower()

# Remove trailing white spaces
income_df['country'] = income_df['country'].str.strip()

# Let us view the data
countries = income_df['country'].unique()
print(f"There are {len(countries)} unique countries")
countries

There are 37 unique countries


array(['united states/', 'britain', 'united states', 'britain/',
       'united kingdom', 'u.k.', 'sa', 'u.k/', 'america', nan, 's.a.',
       'england', 'uk', 's.a./', 'u.k', 'america/', 'sa.', '', 'uk.',
       'england/', 'united states of america', 'uk/', 'sa/', 'england.',
       'america.', 's.a..', 'united states of america.',
       'united states of america/', 'united states.',
       's. africasouth africa', 'britain.', '/', 'united kingdom.',
       's. africasouth africa/', 'united kingdom/',
       's. africasouth africa.', '.'], dtype=object)

2. Note that there should only be three separate countries. Eliminate all variations, so that 'South Africa', 'United Kingdom' and 'United States' are the only three countries.

In [39]:
# Get the top 10 closest matches to "united kingdom"
matches = fuzzywuzzy.process.extract("uk", countries, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

# Inspect matches
matches


[('uk', 100),
 ('uk.', 100),
 ('uk/', 100),
 ('u.k.', 40),
 ('u.k/', 40),
 ('u.k', 40),
 ('united states/', 13),
 ('united states', 13),
 ('united states.', 13),
 ('united kingdom', 12)]

In [85]:
# Function to replace rows in the provided column of the provided DataFrame
# that match the provided string above the provided ratio with the provided string
def replace_matches_in_column(df, column, string_to_match, min_ratio = 40):
    # get a list of unique strings
    strings = df[column].unique()

    # Get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings,
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # Only get matches with a ratio > 90
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

    # Get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # Replace all rows with close matches with the input matches
    df.loc[rows_with_matches, column] = string_to_match

    # Let us know when the function is done
    print("All done!")

In [86]:
replace_matches_in_column(df=income_df, column='country', string_to_match="united states")
replace_matches_in_column(df=income_df, column='country', string_to_match="britain")
replace_matches_in_column(df=income_df, column='country', string_to_match="united kingdom")
replace_matches_in_column(df=income_df, column='country', string_to_match="uk")
replace_matches_in_column(df=income_df, column='country', string_to_match="sa")
replace_matches_in_column(df=income_df, column='country', string_to_match="america")
replace_matches_in_column(df=income_df, column='country', string_to_match="united states of america")
replace_matches_in_column(df=income_df, column='country', string_to_match="england")


All done!
All done!
All done!
All done!
All done!
All done!
All done!
All done!


In [49]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

fuzz.ratio('uk','us')

50

In [89]:
# get all the unique values in the 'country' column
countries = income_df['country'].unique()

print(f"There are {len(countries)} unique countries")
countries


There are 7 unique countries


array(['united states of america', 'uk', 'sa', 'england', '', '/', '.'],
      dtype=object)

In [90]:
income_df.replace('uk', 'united kingdom', inplace=True)
income_df.replace('united states of america', 'united states', inplace=True)
income_df.replace('sa','south africa', inplace=True)
income_df.replace('england', 'united kingdom', inplace=True)

# Get all the unique values in the 'country' column
countries = income_df['country'].unique()

print(f"There are {len(countries)} unique countries")
countries

There are 6 unique countries


array(['united states', 'united kingdom', 'south africa', '', '/', '.'],
      dtype=object)

In [91]:
income_df.drop(income_df.loc[income_df['country']==''].index, inplace=True)
income_df.drop(income_df.loc[income_df['country']=='/'].index, inplace=True)
income_df.drop(income_df.loc[income_df['country']=='.'].index, inplace=True)

In [92]:
# Get all the unique values in the 'country' column
countries = income_df['country'].unique()

print(f"There are {len(countries)} unique countries")
countries

There are 3 unique countries


array(['united states', 'united kingdom', 'south africa'], dtype=object)

3. Create a new column called `days_ago` in the DataFrame that is a copy of the 'date_measured' column but instead it is a number that shows how many days ago it was measured from the current date. Note that the current date can be obtained using `datetime.date.today()`.

In [93]:
# Import modules
from datetime import date

# Get the first few rows of the dataset
income_df.head()


,id,store_name,store_email,department,income,date_measured,country
0,1,"Cullen/Frost Bankers, Inc.",NaN,Clothing,$54438554.24,4-2-2006,united states
1,2,Nordson Corporation,NaN,Tools,$41744177.01,4-1-2006,united states
2,3,"Stag Industrial, Inc.",NaN,Beauty,$36152340.34,12-9-2003,united states
3,4,FIRST REPUBLIC BANK,ecanadine3@fc2.com,Automotive,$8928350.04,8-5-2006,united states
4,5,Mercantile Bank Corporation,NaN,Baby,$33552742.32,21-1-1973,united states


In [100]:
import datetime
income_df['days_ago'] = pd.to_datetime(datetime.date.today()) - pd.to_datetime(income_df['date_measured'], format = '%d-%m-%Y')

In [102]:
income_df

,id,store_name,store_email,department,income,date_measured,country,days_ago
0,1,"Cullen/Frost Bankers, Inc.",NaN,Clothing,$54438554.24,4-2-2006,united states,6658 days
1,2,Nordson Corporation,NaN,Tools,$41744177.01,4-1-2006,united states,6689 days
2,3,"Stag Industrial, Inc.",NaN,Beauty,$36152340.34,12-9-2003,united states,7534 days
3,4,FIRST REPUBLIC BANK,ecanadine3@fc2.com,Automotive,$8928350.04,8-5-2006,united states,6565 days
4,5,Mercantile Bank Corporation,NaN,Baby,$33552742.32,21-1-1973,united states,18725 days
...,...,...,...,...,...,...,...,...
995,996,Columbia Sportswear Company,cschooleyrn@sohu.com,Automotive,$52593924.99,7-10-2005,united states,6778 days
996,997,WisdomTree Interest Rate Hedged High Yield Bon...,NaN,Electronics,$60473676.46,19-12-1990,united states,12184 days
997,998,Tortoise Energy Infrastructure Corporation,cbeardshallrp@ow.ly,Health,$1697293.64,25-4-2009,united states,5482 days
998,999,Qwest Corporation,NaN,Beauty,$30091863.73,13-1-2011,united kingdom,4854 days
